# From Pixels to Latents

**Module 6.3, Lesson 5** | CourseAI

In the lesson, you learned that latent diffusion runs the **same diffusion algorithm** you implemented in Module 6.2, operating on the VAE's compressed 64x64x4 latent tensors instead of 512x512x3 pixel images. Now you will explore this hands-on with Stable Diffusion's actual components.

**What you will do:**
- Load Stable Diffusion's pre-trained VAE and encode/decode real images to see the 48x compression in action
- Visualize the 4 latent channels as heatmaps and interpolate between encoded images in latent space
- Compute the compression ratio and estimate the computational cost savings that make SD practical
- Trace the full encode-diffuse-decode pipeline step by step using real SD components

**For each exercise, PREDICT the output before running the cell.**

This is a CONSOLIDATE notebook. No new algorithms, no new math. Every concept here is something you already know from Modules 6.1, 6.2, and 6.3. The exercises verify that you can connect the pieces and trace data through the full pipeline.

**Estimated time:** 30-45 minutes.

---

## Setup

Run this cell to install dependencies and import everything.

In [ ]:
!pip install -q diffusers transformers accelerate

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# Reproducible results
torch.manual_seed(42)
np.random.seed(42)

# Nice plots
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = [10, 4]

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if device.type == 'cuda':
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB')

print('\nSetup complete.')

## Shared Helpers

Utilities for loading images and converting between PIL images and tensors.

In [ ]:
def load_image_from_url(url, size=512):
    """Download an image from a URL and resize to size x size."""
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert('RGB')
    img = img.resize((size, size), Image.LANCZOS)
    return img


def pil_to_tensor(img):
    """Convert a PIL image to a tensor in the format the VAE expects.
    
    SD's VAE expects: (batch, 3, H, W) with values in [-1, 1].
    PIL images are (H, W, 3) with values in [0, 255].
    """
    arr = np.array(img).astype(np.float32) / 255.0  # [0, 1]
    arr = arr * 2.0 - 1.0  # [-1, 1]
    tensor = torch.from_numpy(arr).permute(2, 0, 1).unsqueeze(0)  # (1, 3, H, W)
    return tensor.to(device)


def tensor_to_pil(tensor):
    """Convert a VAE output tensor back to a PIL image.
    
    VAE output: (batch, 3, H, W) with values roughly in [-1, 1].
    """
    img = tensor.detach().cpu().squeeze(0).permute(1, 2, 0).numpy()
    img = (img + 1.0) / 2.0  # [-1, 1] -> [0, 1]
    img = np.clip(img, 0.0, 1.0)
    return Image.fromarray((img * 255).astype(np.uint8))


# Load sample images -- diverse subjects for comparing latent representations
image_urls = {
    'cat': 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Cat_November_2010-1a.jpg/1200px-Cat_November_2010-1a.jpg',
    'landscape': 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/10/Kluane_National_Park_-_June_2015.jpg/1280px-Kluane_National_Park_-_June_2015.jpg',
    'building': 'https://upload.wikimedia.org/wikipedia/commons/thumb/a/a1/Neues_Rathaus_Hannover_2013.jpg/1024px-Neues_Rathaus_Hannover_2013.jpg',
}

images = {}
for name, url in image_urls.items():
    try:
        images[name] = load_image_from_url(url)
        print(f'Loaded {name}: {images[name].size}')
    except Exception as e:
        print(f'Failed to load {name}: {e}')
        # Fallback: generate a simple test pattern
        arr = np.random.randint(0, 255, (512, 512, 3), dtype=np.uint8)
        images[name] = Image.fromarray(arr)
        print(f'  Using random test pattern for {name}')

print(f'\nLoaded {len(images)} images, all resized to 512x512.')

---

## Exercise 1: Explore SD's VAE Encoder and Decoder [Guided]

In Module 6.1, you built a VAE that compressed 28x28x1 Fashion-MNIST images (784 values) into a 32-dimensional latent code. Stable Diffusion's VAE does the same thing at a much larger scale: it compresses 512x512x3 images (786,432 values) into 64x64x4 latent tensors (16,384 values).

The lesson called the VAE a **translator between two languages**: pixel language (what you see) and latent language (what the diffusion model speaks). Now you will load this translator and use it.

**Before running, predict:**
- What shape will the latent tensor be after encoding a 512x512x3 image? (Think: the lesson said 64x64x4. With a batch dimension, that is `(1, 4, 64, 64)`.)
- How many values is that? And what is the compression ratio compared to the input? (16,384 values vs 786,432 values = 48x compression.)
- Will the decoded image look identical to the original, or slightly different? (Slightly different -- the VAE is lossy compression. But SD's VAE uses perceptual + adversarial loss, so it should be much sharper than your Module 6.1 toy VAE.)

In [ ]:
from diffusers import AutoencoderKL

# Load Stable Diffusion v1.5's VAE
# This is the exact same VAE used in the full SD pipeline.
# It was trained separately on image reconstruction, then FROZEN.
vae = AutoencoderKL.from_pretrained(
    'stable-diffusion-v1-5/stable-diffusion-v1-5',
    subfolder='vae',
    torch_dtype=torch.float16 if device.type == 'cuda' else torch.float32,
)
vae = vae.to(device)
vae.eval()  # Inference mode -- no training, no gradients

print(f'VAE loaded on {device}')
print(f'Parameters: {sum(p.numel() for p in vae.parameters()) / 1e6:.1f}M')

In [ ]:
# Encode and decode the cat image
cat_img = images['cat']
cat_tensor = pil_to_tensor(cat_img)

print('=== Encoding ===')
print(f'Input shape: {cat_tensor.shape}')   # (1, 3, 512, 512)
print(f'Input values: [{cat_tensor.min():.1f}, {cat_tensor.max():.1f}]')
print(f'Input size: {cat_tensor.shape[1]} x {cat_tensor.shape[2]} x {cat_tensor.shape[3]}')
print(f'Input values count: {cat_tensor[0].numel():,}')

with torch.no_grad():
    # The VAE encoder outputs a distribution (mean + logvar), just like your Module 6.1 VAE.
    # .latent_dist gives us the distribution; .sample() applies the reparameterization trick.
    latent_dist = vae.encode(cat_tensor).latent_dist
    latent = latent_dist.sample()
    
    # SD applies a scaling factor to the latents (keeps them in a nice range for diffusion)
    latent = latent * vae.config.scaling_factor

print(f'\nLatent shape: {latent.shape}')    # (1, 4, 64, 64)
print(f'Latent values: [{latent.min():.2f}, {latent.max():.2f}]')
print(f'Latent size: {latent.shape[1]} x {latent.shape[2]} x {latent.shape[3]}')
print(f'Latent values count: {latent[0].numel():,}')

# Compression ratio
input_values = cat_tensor[0].numel()
latent_values = latent[0].numel()
ratio = input_values / latent_values
print(f'\nCompression ratio: {input_values:,} / {latent_values:,} = {ratio:.1f}x')
print(f'The VAE compressed the image by {ratio:.0f}x.')

In [ ]:
# Decode: latent -> pixel space
print('=== Decoding ===')
with torch.no_grad():
    # Undo the scaling factor before decoding
    decoded = vae.decode(latent / vae.config.scaling_factor).sample

print(f'Decoded shape: {decoded.shape}')  # (1, 3, 512, 512)
print(f'Decoded values: [{decoded.min():.2f}, {decoded.max():.2f}]')

reconstructed_img = tensor_to_pil(decoded)

# Compare original vs reconstruction
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(cat_img)
axes[0].set_title('Original (512x512x3)', fontsize=12)
axes[0].axis('off')

# Show the latent as a placeholder (just channel 0)
latent_vis = latent[0, 0].float().cpu().numpy()
axes[1].imshow(latent_vis, cmap='magma')
axes[1].set_title('Latent channel 0 (64x64)', fontsize=12)
axes[1].axis('off')

axes[2].imshow(reconstructed_img)
axes[2].set_title('Reconstructed (512x512x3)', fontsize=12)
axes[2].axis('off')

plt.suptitle('SD VAE: Encode -> Latent (48x smaller) -> Decode', fontsize=13)
plt.tight_layout()
plt.show()

# Compute reconstruction quality (MSE and PSNR)
mse = F.mse_loss(decoded.float(), cat_tensor.float()).item()
psnr = 10 * np.log10(4.0 / mse)  # input range is [-1,1] so max range is 2, max^2 = 4
print(f'\nReconstruction MSE: {mse:.6f}')
print(f'Reconstruction PSNR: {psnr:.1f} dB')
print()
print('Observations:')
print('  1. The reconstruction is very close to the original.')
print('     SD\'s VAE uses perceptual + adversarial loss, NOT just MSE,')
print('     producing much sharper results than your Module 6.1 toy VAE.')
print('  2. The latent is 64x64x4 -- NOT a small image. It is an abstract')
print('     learned representation. Channel 0 shown above has structure,')
print('     but it is NOT a downscaled photo.')
print('  3. The 48x compression is dramatic: 786,432 values -> 16,384 values.')
print('     The diffusion model will process these tiny tensors at every step.')

### What Just Happened

You loaded the exact VAE used in Stable Diffusion and saw the encode-decode pipeline in action:

1. **Encode:** 512x512x3 image (786,432 values) -> 64x64x4 latent (16,384 values). A **48x compression**.
2. **Decode:** 64x64x4 latent -> 512x512x3 reconstructed image. Nearly identical to the original.

The reconstruction quality is dramatically better than your Module 6.1 toy VAE. SD's VAE preserves edges, textures, and fine detail because it uses perceptual and adversarial losses instead of plain MSE. The VAE is a **high-fidelity translator** -- good enough that the diffusion model can work entirely in latent space without quality concerns.

The latent tensor is an abstract representation, not a small image. You cannot interpret it visually as a photo. The decoder is essential -- it translates from the latent language back to pixels.

---

## Exercise 2: Visualize the Latent Space [Guided]

The latent tensor has 4 channels. What do these channels capture? In your Module 6.1 VAE, the 32-dimensional bottleneck was too abstract to interpret individually. SD's latent channels are spatial (64x64), so we can visualize them as heatmaps.

We will also interpolate between two encoded images in latent space -- a direct callback to your **Exploring Latent Spaces** experience in Module 6.1. There, you interpolated between a T-shirt and a sneaker and got coherent intermediates. The same property holds here, at much higher resolution.

**Before running, predict:**
- Will the 4 latent channels look the same or different from each other? (Different -- each channel captures different aspects of the image, similar to how CNN feature maps at different channels capture different patterns.)
- When you interpolate between two encoded images in latent space and decode the intermediates, will you get coherent intermediate images or garbage? (Coherent intermediates -- the VAE's KL regularization organized the latent space. You proved this in Module 6.1. "Clouds, not points.")

In [ ]:
# Encode all three images
latents = {}
for name, img in images.items():
    tensor = pil_to_tensor(img)
    with torch.no_grad():
        dist = vae.encode(tensor).latent_dist
        z = dist.sample() * vae.config.scaling_factor
    latents[name] = z
    print(f'{name}: encoded to {z.shape}')

# Visualize the 4 channels for each image
fig, axes = plt.subplots(len(images), 5, figsize=(18, 3.5 * len(images)))

for row, (name, img) in enumerate(images.items()):
    # Original image
    axes[row, 0].imshow(img)
    axes[row, 0].set_title(f'{name} (original)', fontsize=10)
    axes[row, 0].axis('off')
    
    # 4 latent channels
    z = latents[name]
    for ch in range(4):
        channel_data = z[0, ch].float().cpu().numpy()
        axes[row, ch + 1].imshow(channel_data, cmap='magma')
        axes[row, ch + 1].set_title(f'Channel {ch}', fontsize=10)
        axes[row, ch + 1].axis('off')

plt.suptitle('Latent channels for different images (64x64 each)', fontsize=13)
plt.tight_layout()
plt.show()

print('Observations:')
print('  1. Different channels capture different aspects of the image.')
print('     Some channels emphasize edges, others capture color or brightness.')
print('  2. The channels are NOT RGB. They are abstract learned features.')
print('     The 4-channel representation encodes perceptual content,')
print('     not raw color information.')
print('  3. The spatial structure of the original image is visible in the')
print('     channels -- the cat\'s shape, the mountain\'s outline, the')
print('     building\'s structure. The VAE preserves WHERE things are.')

In [ ]:
# Interpolate between two images in latent space
# This is the same experiment from Module 6.1 (Exploring Latent Spaces),
# now with SD's VAE at 512x512 resolution.

names = list(images.keys())
name_a, name_b = names[0], names[1]  # e.g., cat and landscape
z_a = latents[name_a]
z_b = latents[name_b]

# Linear interpolation in latent space
n_steps = 7
alphas = np.linspace(0, 1, n_steps)

interpolated_images = []
for alpha in alphas:
    z_interp = (1 - alpha) * z_a + alpha * z_b
    with torch.no_grad():
        decoded = vae.decode(z_interp / vae.config.scaling_factor).sample
    interpolated_images.append(tensor_to_pil(decoded))

# Display the interpolation
fig, axes = plt.subplots(1, n_steps, figsize=(20, 3.5))
for i, (img, alpha) in enumerate(zip(interpolated_images, alphas)):
    axes[i].imshow(img)
    if i == 0:
        axes[i].set_title(f'{name_a}\n(alpha=0)', fontsize=9)
    elif i == n_steps - 1:
        axes[i].set_title(f'{name_b}\n(alpha=1)', fontsize=9)
    else:
        axes[i].set_title(f'alpha={alpha:.2f}', fontsize=9)
    axes[i].axis('off')

plt.suptitle(f'Latent space interpolation: {name_a} -> {name_b}', fontsize=13)
plt.tight_layout()
plt.show()

print(f'Interpolation: {name_a} -> {name_b} in {n_steps} steps')
print()
print('Observations:')
print('  1. The intermediates are COHERENT images, not ghostly overlaps.')
print('     This is latent-space interpolation, not pixel-space averaging.')
print('  2. This is the same property you saw in Module 6.1 with Fashion-MNIST:')
print('     the VAE\'s KL regularization organized the latent space so that')
print('     every point decodes to something meaningful.')
print('  3. This continuity is what makes latent diffusion POSSIBLE.')
print('     The denoising trajectory passes through latent space, and at every')
print('     step, the current point must decode to something coherent.')
print('     The "roads" in latent space (from Module 6.1) are what the')
print('     diffusion model walks along during denoising.')

### What Just Happened

You explored SD's latent space in two ways:

1. **Channel visualization:** The 4 latent channels capture different aspects of the image -- edges, brightness, color structure. They are abstract learned features, NOT RGB channels or a downscaled thumbnail. The spatial structure is preserved (you can see shapes), but the representation is qualitatively different from pixels.

2. **Latent interpolation:** Linear interpolation between two encoded images produces coherent intermediate images, not ghostly overlaps. This is the same property you proved in Module 6.1 -- the VAE's KL regularization organizes the latent space so every point decodes to something meaningful.

This continuity is the foundation for latent diffusion. The denoising trajectory is a path through latent space from pure noise (z_T) to a clean latent (z_0). At every point along that path, the latent must be decodable into something coherent. The organized latent space guarantees this. **The VAE built the roads; diffusion walks them.**

---

## Exercise 3: Compute the Compression Ratio and Cost Savings [Supported]

The lesson showed that the VAE compresses 512x512x3 to 64x64x4 -- a 48x reduction. But how does this translate to actual computational savings during diffusion?

Convolution cost scales roughly with spatial area (H x W). The U-Net runs at every denoising step, so the savings multiply by the number of steps. Your task: compute the concrete numbers and connect them to your capstone timing experience.

**Hints:**
- Pixel-space spatial area: 512 x 512 = 262,144
- Latent-space spatial area: 64 x 64 = 4,096
- `spatial_ratio = pixel_area / latent_area`
- For the capstone comparison: your 28x28 MNIST model's spatial area was 784

In [ ]:
# ---- Part 1: Value count and compression ratio ----

# TODO: Compute the number of values in a pixel-space image and a latent tensor.
# Pixel-space: 512 * 512 * 3
# Latent-space: 64 * 64 * 4
pixel_values = None   # TODO: replace with the computation
latent_values = None  # TODO: replace with the computation
compression_ratio = None  # TODO: pixel_values / latent_values

print('=== Value Count ===')
if pixel_values is not None:
    print(f'Pixel-space:  512 x 512 x 3 = {pixel_values:>10,} values')
    print(f'Latent-space:  64 x  64 x 4 = {latent_values:>10,} values')
    print(f'Compression ratio:             {compression_ratio:.1f}x')
else:
    print('Fill in the TODOs above to compute the compression ratio.')

In [ ]:
# ---- Part 2: Spatial area and convolution cost ----
# Convolution cost scales with spatial area (H x W).
# The U-Net processes feature maps at this spatial resolution.

# TODO: Compute the spatial area for pixel-space and latent-space.
pixel_spatial_area = None    # TODO: 512 * 512
latent_spatial_area = None   # TODO: 64 * 64
spatial_ratio = None         # TODO: pixel_spatial_area / latent_spatial_area

print('=== Spatial Area (drives convolution cost) ===')
if pixel_spatial_area is not None:
    print(f'Pixel-space:  512 x 512 = {pixel_spatial_area:>10,} spatial positions')
    print(f'Latent-space:  64 x  64 = {latent_spatial_area:>10,} spatial positions')
    print(f'Spatial area ratio:        {spatial_ratio:.0f}x fewer spatial positions')
    print()
    print(f'The U-Net processes {spatial_ratio:.0f}x fewer spatial positions per step.')
    print(f'Over 50 denoising steps, that is {spatial_ratio:.0f} x 50 = {spatial_ratio * 50:.0f}x')
    print(f'fewer total spatial computations.')
else:
    print('Fill in the TODOs above.')

In [ ]:
# ---- Part 3: Connect to your capstone timing ----
# In Build a Diffusion Model (6.2.5), you timed pixel-space diffusion on 28x28 MNIST.
# How does 512x512 pixel-space compare? And how does 64x64 latent-space compare?

capstone_spatial_area = 28 * 28  # your MNIST images were 28x28 = 784 spatial positions

# TODO: Compute how many times larger 512x512 is compared to 28x28
# Hint: pixel_spatial_area / capstone_spatial_area
pixel_vs_capstone = None  # TODO

# TODO: Compute how many times larger 64x64 is compared to 28x28
# Hint: latent_spatial_area / capstone_spatial_area
latent_vs_capstone = None  # TODO

print('=== Comparison to Your Module 6.2 Capstone ===')
if pixel_vs_capstone is not None and latent_vs_capstone is not None:
    print(f'Your capstone: 28 x 28 = {capstone_spatial_area:,} spatial positions')
    print()
    print(f'512x512 pixel-space: {pixel_vs_capstone:.0f}x larger than your capstone')
    print(f'  If your capstone took T minutes, pixel-space SD would take ~{pixel_vs_capstone:.0f}T minutes.')
    print(f'  For T=2 minutes: ~{pixel_vs_capstone * 2:.0f} minutes ({pixel_vs_capstone * 2 / 60:.0f} hours). Impractical.')
    print()
    print(f'64x64 latent-space: {latent_vs_capstone:.1f}x larger than your capstone')
    print(f'  If your capstone took T minutes, latent-space SD would take ~{latent_vs_capstone:.1f}T minutes.')
    print(f'  For T=2 minutes: ~{latent_vs_capstone * 2:.1f} minutes. Practical!')
    print()
    print(f'The difference between "impractical" ({pixel_vs_capstone:.0f}x your capstone)')
    print(f'and "runs on a consumer GPU" ({latent_vs_capstone:.1f}x your capstone)')
    print(f'is the VAE compression.')
else:
    print('Fill in the TODOs above.')

<details>
<summary>Solution</summary>

The key insight: the compression ratio is about **value count** (48x), but the computational savings for the U-Net are about **spatial area** (64x). Both are dramatic, and both make latent diffusion practical.

**Part 1: Value count**
```python
pixel_values = 512 * 512 * 3       # 786,432
latent_values = 64 * 64 * 4        # 16,384
compression_ratio = pixel_values / latent_values  # 48.0x
```

**Part 2: Spatial area**
```python
pixel_spatial_area = 512 * 512      # 262,144
latent_spatial_area = 64 * 64       # 4,096
spatial_ratio = pixel_spatial_area / latent_spatial_area  # 64.0x
```

Note: the spatial ratio (64x) is different from the compression ratio (48x) because pixels have 3 channels while latents have 4 channels. The spatial ratio is what matters for convolution cost, since convolution scales with H x W.

**Part 3: Capstone comparison**
```python
pixel_vs_capstone = pixel_spatial_area / capstone_spatial_area    # ~335x
latent_vs_capstone = latent_spatial_area / capstone_spatial_area  # ~5.2x
```

If your 28x28 capstone took 2 minutes, pixel-space diffusion at 512x512 would take ~670 minutes (11 hours). Latent-space diffusion at 64x64 would take ~10 minutes. That is the difference between "impractical" and "runs on a consumer GPU."

Common mistake: confusing the compression ratio (48x, based on total values) with the spatial ratio (64x, based on H x W). For convolution cost, spatial area is what matters.

</details>

---

## Exercise 4: Trace the Full Pipeline [Independent]

You now know every component of Stable Diffusion. In this exercise, you will manually execute the encode-diffuse-decode pipeline step by step, using real pre-trained SD components. No `pipe()` convenience function -- you will call each component yourself.

**Your task:**

1. **Encode a text prompt** using CLIP's text encoder (the lesson calls this the "WHAT")
2. **Encode an image** with the VAE encoder to get the starting latent z_0
3. **Add noise** at a specific timestep using the diffusion scheduler (the same forward process formula from Module 6.2)
4. **Run one denoising step** with the U-Net, providing the noisy latent, timestep, and text embeddings
5. **Decode** the result with the VAE decoder to get a pixel image

After completing the pipeline, answer the reflection questions:
- Which step is the "encode" (VAE compression)?
- Which step is the "diffuse" (the part that is identical to Module 6.2)?
- Which step is the "decode" (translation back to pixels)?
- What would happen if you skipped step 2 and started from pure random noise instead? (That is unconditional generation -- the normal SD sampling process.)

**Key tips:**
- All SD v1.5 components live under the same model ID with different subfolders -- look at how the VAE was loaded in Exercise 1 for the pattern
- The `diffusers` and `transformers` libraries provide the U-Net, scheduler, tokenizer, and text encoder -- check their docs for the right classes
- The scheduler has methods for both the forward process (adding noise) and reverse process (one denoising step)
- Use `torch.float16` on GPU for memory efficiency; wrap inference in `torch.no_grad()`
- If you get stuck, the full solution is in the collapsible block below the code cell

In [ ]:
# YOUR CODE HERE
#
# Execute the 5 conceptual steps from the markdown above:
#   1. Encode a text prompt into embeddings (CLIP tokenizer + text encoder)
#   2. Encode an image into the latent space (VAE encoder, already loaded)
#   3. Add noise at a specific timestep (diffusion scheduler)
#   4. Run one U-Net denoising step (provide noisy latent, timestep, text embeddings)
#   5. Decode the result back to pixel space (VAE decoder)
#
# Refer to the diffusers and transformers library docs, or look at how the
# VAE was loaded in Exercise 1 for patterns.
# The full solution is in the <details> block below if you get stuck.


<details>
<summary>Solution</summary>

The key insight: you are manually executing the same pipeline that `StableDiffusionPipeline` automates. Each step maps to a concept from a specific module:

- **Text encoding (Step 4):** Module 6.3, Lessons 3-4 (CLIP + cross-attention)
- **VAE encoding (Step 5):** Module 6.1 (VAE encoder-decoder)
- **Adding noise (Step 6):** Module 6.2 (forward process, closed-form formula)
- **U-Net denoising (Step 7):** Module 6.2 (noise prediction) + Module 6.3 (U-Net architecture, conditioning)
- **VAE decoding (Step 9):** Module 6.1 (decoder translates latent -> pixel)

The diffusion part (steps 6-8) is **identical** to what you implemented in Module 6.2. The only differences: the tensors are 64x64x4 instead of 28x28x1, and the U-Net receives text embeddings via cross-attention.

**Note on dtype:** On GPU with float16, ensure your input tensors match the model's dtype. The `.to(dtype)` cast on `cat_tensor` below is important -- without it, you may get a dtype mismatch between your float32 image tensor and the float16 VAE/U-Net. On CPU (float32 throughout), this is not an issue.

```python
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import UNet2DConditionModel, DDPMScheduler

model_id = 'stable-diffusion-v1-5/stable-diffusion-v1-5'
dtype = torch.float16 if device.type == 'cuda' else torch.float32

# Step 1: Load text encoder and tokenizer
tokenizer = CLIPTokenizer.from_pretrained(model_id, subfolder='tokenizer')
text_encoder = CLIPTextModel.from_pretrained(
    model_id, subfolder='text_encoder', torch_dtype=dtype
).to(device)

# Step 2: Load U-Net
unet = UNet2DConditionModel.from_pretrained(
    model_id, subfolder='unet', torch_dtype=dtype
).to(device)

# Step 3: Load scheduler
scheduler = DDPMScheduler.from_pretrained(model_id, subfolder='scheduler')

# Step 4: Encode text prompt
prompt = "a cat sitting in a garden"
tokens = tokenizer(
    prompt, return_tensors='pt', padding='max_length',
    max_length=77, truncation=True
)
with torch.no_grad():
    text_embeddings = text_encoder(
        tokens.input_ids.to(device)
    ).last_hidden_state  # (1, 77, 768)

print(f'Text embeddings shape: {text_embeddings.shape}')
print(f'  77 token positions, each with a 768-dim embedding.')
print(f'  This is what cross-attention uses as K and V.')

# Step 5: Encode image with VAE
cat_tensor = pil_to_tensor(images['cat']).to(dtype)
with torch.no_grad():
    z_0 = vae.encode(cat_tensor).latent_dist.sample() * vae.config.scaling_factor

print(f'\nLatent z_0 shape: {z_0.shape}')  # (1, 4, 64, 64)
print(f'  This is the ENCODED image in latent space.')

# Step 6: Add noise at timestep 500
noise = torch.randn_like(z_0)
timestep = torch.tensor([500], device=device)
scheduler.set_timesteps(1000)
z_t = scheduler.add_noise(z_0, noise, timestep)

print(f'\nNoisy latent z_t shape: {z_t.shape}')
print(f'  Same formula from Module 6.2:')
print(f'  z_t = sqrt(alpha_bar_t) * z_0 + sqrt(1-alpha_bar_t) * noise')

# Step 7: One U-Net denoising step
with torch.no_grad():
    noise_pred = unet(
        z_t, timestep,
        encoder_hidden_states=text_embeddings
    ).sample

print(f'\nPredicted noise shape: {noise_pred.shape}')  # (1, 4, 64, 64)
print(f'  The U-Net predicts the noise to subtract.')
print(f'  Same as Module 6.2 -- the model predicts epsilon.')

# Step 8: Reverse step
step_output = scheduler.step(noise_pred, timestep.item(), z_t)
z_denoised = step_output.prev_sample

print(f'\nDenoised latent shape: {z_denoised.shape}')
print(f'  One step closer to z_0. In full sampling, you would')
print(f'  repeat this for all timesteps from T down to 0.')

# Step 9: Decode with VAE
with torch.no_grad():
    decoded_noisy = vae.decode(z_t / vae.config.scaling_factor).sample
    decoded_denoised = vae.decode(z_denoised / vae.config.scaling_factor).sample

# Step 10: Display
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(images['cat'])
axes[0].set_title('Original image', fontsize=11)
axes[0].axis('off')

axes[1].imshow(tensor_to_pil(decoded_noisy))
axes[1].set_title(f'Noisy (t=500, decoded)', fontsize=11)
axes[1].axis('off')

axes[2].imshow(tensor_to_pil(decoded_denoised))
axes[2].set_title('After one denoise step (decoded)', fontsize=11)
axes[2].axis('off')

plt.suptitle('Encode -> Add Noise -> Denoise (1 step) -> Decode', fontsize=13)
plt.tight_layout()
plt.show()

print('\n=== Pipeline Summary ===')
print(f'1. Text encoding (CLIP):    prompt -> {text_embeddings.shape}')
print(f'2. Image encoding (VAE):    {cat_tensor.shape} -> {z_0.shape}')
print(f'3. Add noise (forward):     {z_0.shape} -> {z_t.shape} at t=500')
print(f'4. Denoise (U-Net):         {z_t.shape} -> noise pred {noise_pred.shape}')
print(f'5. Reverse step:            {z_t.shape} -> {z_denoised.shape}')
print(f'6. Decode (VAE):            {z_denoised.shape} -> {decoded_denoised.shape}')
print()
print('Which parts are identical to Module 6.2?')
print('  Steps 3, 4, 5: adding noise, predicting noise, reverse step.')
print('  The ALGORITHM is the same. The tensors are 64x64x4 instead of 28x28x1.')
print('  The U-Net also receives text embeddings (cross-attention from Module 6.3).')
print()
print('What is NEW compared to Module 6.2?')
print('  Steps 1, 2, 6: text encoding, VAE encoding, VAE decoding.')
print('  These are the "bookends" -- the VAE translates between pixel and latent space.')
print('  The text encoder provides the WHAT signal via cross-attention.')
```

Note: one denoising step at t=500 will NOT produce a clean image. In full SD sampling, you would loop from t=T down to t=0 (typically 50 steps with DDIM). The point here is to trace the pipeline, not to generate a finished image.

If you started from pure random noise (skipping step 5), that is exactly what normal SD generation does: sample z_T from N(0,I), then denoise for 50 steps, then decode. The image encoding step is only used for image-to-image generation or for exercises like this one.

</details>

---

## Key Takeaways

1. **SD's VAE compresses 512x512x3 images to 64x64x4 latents -- a 48x reduction in values, 64x reduction in spatial area.** The reconstruction quality is high because SD's VAE uses perceptual + adversarial losses, not plain MSE. The VAE is a high-fidelity translator between pixel space and latent space.

2. **The latent space is organized and continuous, just like your Module 6.1 VAE.** Interpolation produces coherent intermediates. Every point decodes to something meaningful. This is what makes latent diffusion possible -- the denoising trajectory stays in meaningful territory at every step. The VAE built the roads; diffusion walks them.

3. **The computational savings are dramatic and concrete.** 64x fewer spatial positions per denoising step. Your 28x28 capstone was ~5x smaller than latent space (manageable) but ~335x smaller than pixel space at 512x512 (impractical). The VAE compression is the difference between "runs on a consumer GPU" and "requires a data center."

4. **The full pipeline has clear stages: encode (CLIP + VAE) -> diffuse (identical to Module 6.2) -> decode (VAE).** Every formula from Module 6.2 applies unchanged. The noise schedule, the forward process, the reverse step, the training loss -- all the same. Only the tensor dimensions and the addition of text conditioning via cross-attention are different.

5. **That is Stable Diffusion.** VAE for compression, diffusion for generation, U-Net for multi-scale denoising, sinusoidal embeddings for timestep, cross-attention for text, CFG for text amplification, latent space for speed. Every piece solves a specific problem. You understand all of them.